### Classifying Covid tweets using the FastAI library and Universal Language Model Fine-tuning for Text Classification (ULMFiT)

#### Please give an upvote if you like this simple kernel.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from pathlib import Path

path = Path('../input/covid-19-nlp-text-classification/')

train = pd.read_csv(path / 'Corona_NLP_train.csv', encoding = 'latin1')
test = pd.read_csv(path / 'Corona_NLP_test.csv', encoding = 'latin1')

In [ ]:
train.head()

In [ ]:
train['Sentiment'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize = (8, 8))
sns.countplot(x = train['Sentiment'] , palette = 'viridis')
plt.show()

In [ ]:
train.head()

In [ ]:
train = train.drop(columns = ['UserName', 'ScreenName', 'Location', 'TweetAt'])

In [ ]:
train.reset_index(drop=True, inplace=True)
train.head()

In [ ]:
from fastai.text.all import *

In [ ]:
dls = TextDataLoaders.from_df(train, valid_pct = 0.1, text_col = 'OriginalTweet', label_col = 'Sentiment', bs = 64, is_lm = True)

In [ ]:
dls.show_batch()

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult = 0.3)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, lr_max = slice(1e-2, 1e-1))

In [ ]:
learn.save('fine_tuned')
learn.save_encoder('fine_tuned_enc')

In [ ]:
TEXT = "This pandemic is destroying our lives"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

In [ ]:
data_class = TextDataLoaders.from_df(train, valid_pct = 0.1, text_col = 'OriginalTweet', label_col = 'Sentiment', bs = 64, text_vocab = dls.vocab)

In [ ]:
data_class.show_batch()

In [ ]:
learn = text_classifier_learner(data_class, AWD_LSTM, drop_mult=0.5, metrics=[accuracy])
learn.load_encoder('fine_tuned_enc')

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 1e-3)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(k = 5)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
learn.predict("This pandemic is destroying our lives")

In [ ]:
learn.predict("Let us all be hopeful for a new tomorrow!")

The last year has been rough but as Helen Keller says "Keep your face to the sunshine and you shall never see a shadow", we should all be hopeful for the new dawn. 
![](https://www.finsyn.com/wp-content/uploads/2020/03/It%E2%80%99s-Always-Darkest-Before-the-Dawn-845x325.jpg)

Please Give an upvote if you like this notebook.

Keep learning, keep sharing!